In [277]:
import open3d as o3d
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

def show_xyz_with_color(xyz_path):
    """XYZファイルを読み込んで、インデックス順に色を付けて表示"""
    path = Path(xyz_path)
    print(f"Reading: {path}")
    pcd = o3d.io.read_point_cloud(str(path), format='xyz')

    points = np.asarray(pcd.points)
    if len(points) == 0:
        raise ValueError("点群が空です。")

    # カラーマップ（インデックスに応じた色）
    cmap = plt.get_cmap("turbo")
    colors = cmap(np.linspace(0, 1, len(points)))[:, :3]  # RGBのみ
    pcd.colors = o3d.utility.Vector3dVector(colors.astype(np.float32))

    # 表示
    o3d.visualization.draw_geometries(
        [pcd],
        window_name=f"Viewing {path.name}",
        width=960, height=720,
        point_show_normal=False,
    )

# 使い方
# show_xyz_with_color("/root/development/project4010/VAE/scripts/t_pose_vertices.xyz")
show_xyz_with_color("/root/development/project4010/gaussian-splatting/pointcloud_xyz/pre_iter_00000.xyz")
# show_xyz_with_color("/root/development/project4010/gaussian-splatting/pointcloud_xyz/in_iter_00100.xyz")
# show_xyz_with_color("/root/development/project4010/gaussian-splatting/pointcloud_xyz/out_iter_07000.xyz")



Reading: /root/development/project4010/gaussian-splatting/pointcloud_xyz/pre_iter_00000.xyz


MESA: error: ZINK: failed to choose pdev
glx: failed to create drisw screen


In [4]:
import open3d as o3d
import numpy as np

# 1. XYZファイルを読み込む
file_path = 't_pose_vertices.xyz' 

try:
    vertices = np.loadtxt(file_path, dtype=np.float32)
    print(f"'{file_path}' から {vertices.shape[0]} 個の頂点座標を読み込みました。")
except FileNotFoundError:
    print(f"エラー: ファイル '{file_path}' が見つかりません。パスを確認してください。")
    exit()

# 2. Open3DのPointCloudオブジェクトを作成
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(vertices)

# 3. 頂点に色を割り当てる (指定範囲のみ色を変更)
num_vertices = vertices.shape[0]
colors = np.zeros((num_vertices, 3)) # 各頂点の色を格納する配列 (N, 3)

# --- 色の設定 ---
# 全ての頂点のデフォルトの色 (例: 灰色)
default_color = [0.7, 0.7, 0.7] # R, G, B (0.0 - 1.0)

# ハイライトする色 (例: 明るい赤)
highlight_color = [1.0, 0.2, 0.2] # R, G, B (0.0 - 1.0)

# ハイライトするインデックスの範囲を指定
# 例として、腕のあたり（適当な範囲）を想定
highlight_start_index = 4000
highlight_end_index = 6500

# まず全ての頂点をデフォルトの色で初期化
for i in range(num_vertices):
    colors[i] = default_color

# 指定した範囲のインデックスの頂点だけ色を変更
for i in range(num_vertices):
    if highlight_start_index <= i < highlight_end_index:
        colors[i] = highlight_color

# 計算した色をPointCloudオブジェクトに割り当てる
pcd.colors = o3d.utility.Vector3dVector(colors)

# 4. Open3Dで可視化
print("Open3Dで可視化を開始します。")
o3d.visualization.draw_geometries([pcd], 
                                  window_name=f"T-Pose Vertices with Highlighted Range ({highlight_start_index}-{highlight_end_index})",
                                  width=800, 
                                  height=600)

print("可視化が終了しました。")

't_pose_vertices.xyz' から 6890 個の頂点座標を読み込みました。
Open3Dで可視化を開始します。
可視化が終了しました。


MESA: error: ZINK: failed to choose pdev
glx: failed to create drisw screen


In [27]:
import open3d as o3d
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt

def convert_ply_to_xyz(ply_path, xyz_path):
    """PLY を読み込み、XYZ 形式で保存する（色情報などは失われる）"""
    print(f"Reading PLY: {ply_path}")
    pcd = o3d.io.read_point_cloud(ply_path, remove_nan_points=True, remove_infinite_points=True)

    if len(pcd.points) == 0:
        raise ValueError("点群が空です。PLYの読み込みに失敗している可能性があります。")

    print(f"Saving XYZ to: {xyz_path}")
    o3d.io.write_point_cloud(xyz_path, pcd, write_ascii=True)  # xyz形式では write_ascii=True を指定すると中身が見やすい

def show_xyz_with_color(xyz_path):
    """XYZファイルを読み込んで、インデックス順に色を付けて表示"""
    path = Path(xyz_path)
    print(f"Reading: {path}")
    pcd = o3d.io.read_point_cloud(str(path), format='xyz')

    points = np.asarray(pcd.points)
    if len(points) == 0:
        raise ValueError("点群が空です。")

    # カラーマップ（インデックスに応じた色）
    cmap = plt.get_cmap("turbo")
    colors = cmap(np.linspace(0, 1, len(points)))[:, :3]  # RGBのみ
    pcd.colors = o3d.utility.Vector3dVector(colors.astype(np.float32))

    # 表示
    o3d.visualization.draw_geometries(
        [pcd],
        window_name=f"Viewing {path.name}",
        width=960, height=720,
        point_show_normal=False,
    )


# -------- 実行部分 --------

# ファイルパスの設定
ply_path = "/root/development/project4010/gaussian-splatting/output/8d6fe5cd-2/point_cloud/iteration_30000/point_cloud.ply"
xyz_path = "/root/development/project4010/gaussian-splatting/pointcloud_xyz/8d6fe5cd-2iter_30000.xyz"

# 出力ディレクトリが存在しない場合は作成
Path(xyz_path).parent.mkdir(parents=True, exist_ok=True)

# 変換 → 表示
convert_ply_to_xyz(ply_path, xyz_path)
show_xyz_with_color(xyz_path)


Reading PLY: /root/development/project4010/gaussian-splatting/output/8d6fe5cd-2/point_cloud/iteration_30000/point_cloud.ply
Saving XYZ to: /root/development/project4010/gaussian-splatting/pointcloud_xyz/8d6fe5cd-2iter_30000.xyz
Reading: /root/development/project4010/gaussian-splatting/pointcloud_xyz/8d6fe5cd-2iter_30000.xyz


MESA: error: ZINK: failed to choose pdev
glx: failed to create drisw screen


In [ ]:
import torch
import smplx
import numpy as np
from smplx.body_models import SMPL
import os, glob, random, inspect

# ---------- NumPy 互換 (NumPy 1.24+) ----------
np.bool   = bool   # type: ignore
np.int    = int    # type: ignore
np.float  = float  # type: ignore
np.complex = complex  # type: ignore
np.object = object # type: ignore
np.unicode = str   # type: ignore
np.str    = str   # type: ignore
if not hasattr(inspect, 'getargspec'):
    inspect.getargspec = inspect.getfullargspec  # type: ignore

# SMPLモデルのパスを指定します。
# ご自身の環境に合わせてパスを変更してください。
# 例えば、ダウンロードしたSMPLモデルのmale/female/neutralのいずれかのモデルファイル（.npz形式）
# のあるディレクトリを指定します。
# 例: model_path = './smpl_models/smpl/'
model_path = "/root/development/project4010/VAE/models" 

# モデルタイプを指定します (例: 'smpl', 'smplh', 'smplx')
# TポーズはSMPLモデルで一般的に定義されますが、smplxライブラリでSMPLモデルをロードできます。
model_type = 'smpl' 

# モデルの性別を指定します ('male', 'female', 'neutral')
# Tポーズの形状は性別によって微妙に異なりますが、ここでは'neutral'を使用することが多いです。
gender = 'NEUTRAL' 

# SMPLモデルをロード
# batch_sizeは1とします。
smpl_model = SMPL(model_path=model_path, gender=gender, batch_size=1)

# Tポーズを作成するためのパラメータを設定
# global_orient: グローバルな回転。ゼロに設定すると、回転なし。
global_orient = torch.zeros(1, 3, device='cpu')

# body_pose: 各関節のポーズ。ゼロに設定すると、ニュートラルポーズ（Tポーズ）。
# SMPLのbody_poseは通常23関節 * 3軸回転 = 69次元です。
body_pose = torch.zeros(1, 69, device='cpu') 

# betas: 身体の形状パラメータ。ゼロに設定すると、平均的な形状。
betas = torch.zeros(1, 10, device='cpu')

# transl: モデルの平行移動。ゼロに設定すると、原点に位置。
transl = torch.zeros(1, 3, device='cpu')

# SMPLモデルにパラメータを入力して、頂点座標を計算
output = smpl_model(
    global_orient=global_orient,
    body_pose=body_pose,
    betas=betas,
    transl=transl,
    return_verts=True # 頂点座標を返すように設定
)

# 頂点座標 (vertices) を取得
# output.verticesは(batch_size, num_vertices, 3)のテンソルです。
# 今回はbatch_size=1なので、1つ目の要素を取り出します。
t_pose_vertices = output.vertices[0].detach().cpu().numpy()

# 関節点の座標 (joints) を取得することもできます（オプション）
# output.jointsは(batch_size, num_joints, 3)のテンソルです。
t_pose_joints = output.joints[0].detach().cpu().numpy()


print("Tポーズの頂点座標の形状:", t_pose_vertices.shape)
# Tポーズの頂点座標の最初の5点を表示
print("Tポーズの頂点座標（最初の5点）:\n", t_pose_vertices[:5])

print("\nTポーズの関節座標の形状:", t_pose_joints.shape)
# Tポーズの関節座標の最初の5点を表示
print("Tポーズの関節座標（最初の5点）:\n", t_pose_joints[:5])

output_filename_txt = 't_pose_vertices.xyz'

# NumPyの savetxt を使用して保存
# fmt='%.4f' は小数点以下4桁まで保存することを意味します
np.savetxt(output_filename_txt, t_pose_vertices, fmt='%.4f')

print(f"Tポーズの頂点座標が '{output_filename_txt}' に保存されました。")

Tポーズの頂点座標の形状: (6890, 3)
Tポーズの頂点座標（最初の5点）:
 [[ 4.4873040e-02  4.9407476e-01  8.9620613e-02]
 [ 3.9471768e-02  4.8138341e-01  9.9601053e-02]
 [ 5.0010335e-02  4.7607607e-01  9.0992793e-02]
 ...
 [ 8.7568603e-02  4.3715644e-01 -5.1905040e-04]
 [ 7.7521287e-02  4.3949991e-01  2.7529825e-04]
 [ 8.5304916e-02  4.4123113e-01  7.7467849e-03]]

Tポーズの関節座標の形状: (45, 3)
Tポーズの関節座標（最初の5点）:
 [[-0.00179507 -0.22333345  0.02821913]
 [ 0.06772467 -0.31473967  0.02140379]
 [-0.06946553 -0.3138551   0.0238993 ]
 [-0.00432793 -0.11437022  0.00152281]
 [ 0.10200122 -0.68993825  0.01690799]]
Tポーズの頂点座標が 't_pose_vertices.xyz' に保存されました。


In [2]:
import pickle
import numpy as np
import os, glob, random, inspect

# ---------- NumPy 互換 (NumPy 1.24+) ----------
np.bool   = bool   # type: ignore
np.int    = int    # type: ignore
np.float  = float  # type: ignore
np.complex = complex  # type: ignore
np.object = object # type: ignore
np.unicode = str   # type: ignore
np.str    = str   # type: ignore
if not hasattr(inspect, 'getargspec'):
    inspect.getargspec = inspect.getfullargspec  # type: ignore

# SMPLモデルファイルのパス
smpl_file_path = '/root/development/project4010/VAE/models/SMPL_NEUTRAL.pkl'

try:
    # ファイルをバイナリ読み込みモードで開く
    # encoding='latin1' は、Python 2で作成されたpickleファイルを読み込む際に必要です。
    with open(smpl_file_path, 'rb') as f:
        smpl_model = pickle.load(f, encoding='latin1')

    # 'f' キーで面のリストを取得
    # SMPLモデルは13776個の面で構成されています。
    faces = smpl_model['f']

    # 結果の表示
    print(f"ファイル '{smpl_file_path}' の読み込みに成功しました。")
    print("-" * 30)
    print(f"面の数: {len(faces)}")
    print(f"面のリストのデータ型: {type(faces)}")
    # numpy arrayの場合、形状も表示すると分かりやすいです
    if isinstance(faces, np.ndarray):
        print(f"面のリストの形状: {faces.shape}")
    print("-" * 30)
    
    # 面のリストの最初の5件を表示
    print("面のリスト（最初の5件）:")
    print(faces[:5])

    # 面のリストの最後の5件を表示
    print("\n面のリスト（最後の5件）:")
    print(faces[-5:])


except FileNotFoundError:
    print(f"エラー: ファイルが見つかりません。パスを確認してください。")
    print(f"指定されたパス: {smpl_file_path}")

except KeyError:
    print(f"エラー: ファイル内にキー 'f' が見つかりませんでした。")
    print("このファイルは有効なSMPLモデルではない可能性があります。")

except Exception as e:
    print(f"予期せぬエラーが発生しました: {e}")

/tmp/ipykernel_1760/1826681347.py:23: DeprecationWarning: Please use `csc_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csc` namespace is deprecated.
  smpl_model = pickle.load(f, encoding='latin1')


ファイル '/root/development/project4010/VAE/models/SMPL_NEUTRAL.pkl' の読み込みに成功しました。
------------------------------
面の数: 13776
面のリストのデータ型: <class 'numpy.ndarray'>
面のリストの形状: (13776, 3)
------------------------------
面のリスト（最初の5件）:
[[1 2 0]
 [0 2 3]
 [2 1 4]
 [4 1 5]
 [2 6 3]]

面のリスト（最後の5件）:
[[3158 4326 4329]
 [1325 3511 4805]
 [4805 3511 6309]
 [3511 1330 6309]
 [6309 1330 4687]]
